In [2]:
#IMporting packages
import os as os
import pandas as pd
#from sqlalchemy import create_engine
#import pymysql
#import mysql.connector

#loading in the nfl team data frame
data_dir = os.path.join(os.getcwd(),"archive (2)")
data_file = os.path.join(data_dir, 'nfl_teams.csv')
nfl_teams = pd.read_csv(data_file)

#loading in the nfl stadiums data frame
data_dir = os.path.join(os.getcwd(),"archive (2)")
data_file2 = os.path.join(data_dir, 'nfl_stadiums.csv')
nfl_stadiums = pd.read_csv(data_file2,encoding='ISO-8859-1')

#loading in the spread of games and scores
data_dir = os.path.join(os.getcwd(),"archive (2)")
data_file3 = os.path.join(data_dir, 'spreadspoke_scores.csv')
spread = pd.read_csv(data_file3, header=0)



#cleaning the data frames so we can import them into sql
#selecting the desired years we want
desired_years = [2017,2018,2019,2020]
spread = spread[spread['schedule_season'].isin(desired_years)]
#removing the weather detail column from spread
spread = spread.drop('weather_detail', axis=1)
#Removing weather because we are bringing our own weather in using an api
spread = spread[['schedule_date',"schedule_season","schedule_week","schedule_playoff","team_home","score_home","score_away","team_away","team_favorite_id","spread_favorite","over_under_line","stadium","stadium_neutral"]]
#making the old team names into the new team names so their is an even 32 teams over the data set
#for the home team column
spread['team_home'] = spread['team_home'].replace('San Diego Chargers', 'Los Angeles Chargers')
spread['team_home'] = spread['team_home'].replace('Washington Football Team', 'Washington Commanders')
spread['team_home'] = spread['team_home'].replace('Washington Redskins', 'Washington Commanders')
#for the away team column
spread['team_away'] = spread['team_away'].replace('San Diego Chargers', 'Los Angeles Chargers')
spread['team_away'] = spread['team_away'].replace('Washington Football Team', 'Washington Commanders')
spread['team_away'] = spread['team_away'].replace('Washington Redskins', 'Washington Commanders')
#print(spread.head())
#num_rows = spread.shape[0]
#print(num_rows)
#exporting the data frame to see if it is correct
#spread.to_csv("spread_final.csv", index=False)



#cleaning the nfl temas csv file
nfl_teams = nfl_teams[["team_name","team_name_short","team_id","team_conference","team_division"]]
nfl_teams = nfl_teams.dropna(subset=['team_division'])
nfl_teams = nfl_teams.drop(nfl_teams[nfl_teams['team_name'] == 'San Diego Chargers'].index)
nfl_teams = nfl_teams.drop(nfl_teams[nfl_teams['team_name'] == 'Washington Football Team'].index)
nfl_teams = nfl_teams.drop(nfl_teams[nfl_teams['team_name'] == 'Washington Redskins'].index)
#num_rows = nfl_teams.shape[0]
#print(num_rows)
#nfl_teams.to_csv("nfl_teams_final.csv", index=False)


#cleaning the nfl stadium data frame
nfl_stadiums = nfl_stadiums[["stadium_name","stadium_location","stadium_open","stadium_close","stadium_type","stadium_capacity","stadium_surface","stadium_latitude","stadium_longitude"]]
#nfl_stadiums.to_csv("nfl_stadiums_final.csv", index=False)


#all of the completed data frames
print(nfl_teams.head())
print(nfl_stadiums.head())
print(spread.head())

           team_name team_name_short team_id team_conference team_division
0  Arizona Cardinals       Cardinals     ARI             NFC      NFC West
1    Atlanta Falcons         Falcons     ATL             NFC     NFC South
3   Baltimore Ravens          Ravens     BAL             AFC     AFC North
5      Buffalo Bills           Bills     BUF             AFC      AFC East
6  Carolina Panthers        Panthers     CAR             NFC     NFC South
        stadium_name  stadium_location  stadium_open  stadium_close  \
0   Acrisure Stadium    Pittsburgh, PA        2001.0            NaN   
1         Alamo Dome   San Antonio, TX           NaN            NaN   
2  Allegiant Stadium      Paradise, NV        2020.0            NaN   
3      Allianz Arena   Munich, Germany           NaN            NaN   
4     Alltel Stadium  Jacksonville, FL           NaN            NaN   

  stadium_type stadium_capacity stadium_surface  stadium_latitude  \
0      outdoor           65,500           Grass       

This is transfering over the data from the data frame into the sql data bases

In [1]:
#!pip install sqlalchemy
#!pip install mysql.connector
#!pip install PyMySQL
#!pip install sqlalchemy
#!pip install mysql-connector-python
#import os
#import pymysql
#import mysql.connector
#from sqlalchemy import create_engine
!pip install mysql-connector-python
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

establishing the connection to the db 

In [4]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "root"
pwd = "Uva!1819"
db_name = "project1_db"

function to load the csv files into the data frames in sql

In [5]:
def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

making the tables in sql through python

In [6]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

In [11]:
dst_dbname="project1_db"
db_operation = "insert"
tables = [('teams', nfl_teams, 'team_name'),
          ('stadiums', nfl_stadiums, 'stadium_name')]
#('spread', spread, 'customer_key'),


In [12]:
engine = create_engine(f'mysql+mysqlconnector://{user_id}:{pwd}@{host_name}/{db_name}')
nfl_teams.to_sql(name='teams', con=engine, if_exists='replace', index=False)

# Close the database connection
connection.close()

C:\Miniconda\envs\py38_default\lib\site-packages\mysql\connector\abstracts.py:130: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if group is 'connector_python':
C:\Miniconda\envs\py38_default\lib\site-packages\mysql\connector\abstracts.py:130: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if group is 'connector_python':


NotSupportedError: (mysql.connector.errors.NotSupportedError) Authentication plugin 'caching_sha2_password' is not supported
(Background on this error at: https://sqlalche.me/e/20/tw8g)